### Making a simple plots using `matplotlib`

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
z = np.loadtxt('data/mt_st_helens_before.dat')
z2 = np.loadtxt('data/mt_st_helens_after.dat')

In [ ]:
#let's inspect the file
z

#### set masks for no-data values

In [ ]:
z_mask = np.ma.masked_array(z, mask=(z==0))
z2_mask = np.ma.masked_array(z2, mask=(z2==0))
nrow, ncol = z.shape

In [ ]:
#let's inspect the file again
z

### quick and dirty plot of an array using ```plt.imshow```

In [ ]:
plt.imshow(z)
plt.colorbar()

#### make an x-y plot using the basic pyplot interface using ```plt.plot()```

In [ ]:
plt.plot(range(0, ncol), z[200, :], label='pre-eruption')
plt.plot(range(0, ncol), z2[200, :], label='post-eruption')
plt.xlabel('Column across raster')
plt.ylabel('Elevation in m')
plt.title('Mt. St. Helens')
plt.legend()

### the matplotlib `axes` object, a gateway to data visualizations

here we create the same plot but first initialize a `fig` and `ax` object from ```plt.subplots()``` that we then add the plotting elements to it the `ax` object

In [ ]:
fig, ax = plt.subplots() #if you leave args blank, it will assume a single plot
ax.plot(range(0, ncol), z[200, :], label='pre-eruption')
ax.plot(range(0, ncol), z2[200, :], label='post-eruption')
ax.set_xlabel('Column across raster')
ax.set_ylabel('Elevation in m')
ax.set_title('Mt. St. Helens')
ax.legend() 

#same same, but different! using an axes object (ax) opens the door to much more plotting flexibility

#### the `AxesSubplot` interface

In [ ]:
fig, ax = plt.subplots(2, 2, sharex=True, sharey=True)

In [ ]:
ax

In [ ]:
ax[0]

#### convert x-axis to meters

In [ ]:
dist = np.linspace(0, 5*5280*.3048, ncol) #each cell is 5 miles, so we need to convert to ft then meters
dist

#### Make a plot with a specified vertical exaggeration of 2x using ```ax.set_aspect()```

In [ ]:
ve = 2
fig, ax = plt.subplots(figsize=(11, 8.5))
ax.plot(dist, z[200, :], label='pre-eruption')
ax.plot(dist, z2[200, :], label='post-eruption')
ax.set_xlabel('Distance in miles')
ax.set_ylabel('Elevation in m')
ax.set_title('Mt. St. Helens')
ax.set_aspect(ve)
ax.text(0.98, .65, 'Vertical Exaggeration: {:.0f}'.format(ve),
        transform=ax.transAxes, ha='right', va='bottom')
xticks = np.arange(0, 6)
ax.set_xticks(xticks*1609.) #m to miles
ax.set_xticklabels(xticks)
ax.tick_params(direction='in')
ax.legend(title='Explanation')
plt.savefig('mtsthelens.pdf', dpi=300, bbox_inches='tight')

The `plt.rcParams` dictionary can be modified to change default matplotlib settings for the current session. For example, changing the default font to a serif font-type:

In [ ]:
sansserif = plt.rcParams['font.family']
plt.rcParams['font.family'] = 'serif'
# plt.rcParams['font.size'] = 10

In [ ]:
plt.subplots()
plt.title('stuff', fontsize=20)

#### make a meshgrid
of the x and y coorindates of each pixel in the DEM

In [ ]:
x = dist
y = np.arange(0, nrow)*x[1]
X, Y = np.meshgrid(x, y)

In [ ]:
X.shape, Y.shape

#### make a pcolormesh overlain with contours

In [ ]:
fig, ax = plt.subplots(figsize=(11, 8.5))
cs = ax.contour(X, Y, np.flipud(z2), colors='k', 
           levels=np.arange(700, 3000, 100), zorder=11)
qm = ax.pcolormesh(X, Y, np.flipud(z2), zorder=-1)
cb = plt.colorbar(qm, label='Elevation, in m')
ax.clabel(cs, inline=1, fontsize=10, fmt='%1.0f')

### let's grab streamflow data for rio grande at embudo and plot!

In [ ]:
import dataretrieval.nwis as nwis
import datetime

In [ ]:
today = datetime.datetime.today().strftime('%Y-%m-%d') #that last bit makes sure our date is in the right format
today

In [ ]:
# specify the USGS site code for which we want data.
site = '08279500'

# # get daily values (dv)
df = nwis.get_record(sites=site, service='dv', start='1889-01-01', end=today)

#did you know the longest continuous streamflow record is here in new mexico!? rio grande at embudo!

In [ ]:
#let's look at it 
df

### what the heck are all of these columns!?

They are code identifiers for different parameters. For example, 00010_Mean is daily mean water temperature in degrees Celsius, and 00060_Mean is daily mean streamflow in cubic feet per second.

You can query the codes here:
https://help.waterdata.usgs.gov/code/parameter_cd_nm_query?parm_nm_cd=%25discharge%25&fmt=html

Let's plot streamflow. Below is the basic plotting recipe that will get the job done! Anything after that is extra...

In [ ]:
#first initialize plot
fig, ax = plt.subplots()

#then plot streamflow as a line. the datetime is the pandas index, 
ax.plot(df.index,df['00060_Mean'])

#now let's add some annotations to the axes and plot
ax.set_ylabel('Discharge (cfs)')

ax.set_xlabel('Year')

ax.set_title('Rio Grande at Embudo')

In [ ]:
#we can easily flip to log-axis, add a grid in the plot background, and change the color and line width

#first initialize plot
fig, ax = plt.subplots()

#then plot streamflow as a line. the datetime is the pandas index, 
ax.plot(df.index,df['00060_Mean'],color='purple',lw=.5,linestyle='--')

#now let's add some annotations to the axes and plot
ax.set_ylabel('Discharge (cfs)')

ax.set_xlabel('Year')

#log scale
ax.set_yscale('log')

#add grid
plt.grid(True)

ax.set_title('Rio Grande at Embudo')